In [2]:
import argparse
import json
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, hooks
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, print_csv_format

import os
import json
import yaml
import torch
import random
import detectron2
import pandas as pd

import cv2 
import numpy as np
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from train import get_dataset_dicts, get_dataset_dicts_with_regions, random_visu, setup_cfg, MyTrainer, load_json_arr, find_best_model

import shutil

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [3]:
### FOLDER PATH ###

dataset_eval = "test"
dataset_eval_output = "val_output_oil_gas_data"
annotation_json = "via_region_data_oil_gas_annotations.json"
results_file = "results.json"
experiment = "../Satellite/output/lr001_BS4_oil_gas_test__aug_1"
conf_path = "../Satellite/output/lr001_BS4_oil_gas_test__aug_1/NSOD2cfg_1000_169r_1000pix_noBT_lr001_BS4_oil_gas_test__aug_1.yaml"
model_path = experiment + "/model_45.pth"
filter_empty_annot = False


annotation_output = os.path.splitext(annotation_json)[0]
out_dir = os.path.join(experiment, dataset_eval_output, annotation_output)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print ("The test directory is created!")

The test directory is created!


In [4]:
def get_dataset_dicts(img_dir, annotation_json):
    with open(os.path.join(img_dir, annotation_json)) as f:
        annotations = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(annotations.values()):
        record = {}
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        annos = v["regions"]
        objs = []
        for anno_key, anno in annos.items():
            shape_attr = anno["shape_attributes"]
            region_attr = anno["region_attributes"]
            
            # Ensure that region_attributes is processed correctly
            if "type" not in region_attr or region_attr["type"] != "oil/gas":
                continue  # Skip non-container regions if any

            px = shape_attr["all_points_x"]
            py = shape_attr["all_points_y"]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,  # Assuming 'container' is the only category
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [5]:
### Indicate to D2 how to read dataset if not in coco format: ###

# D2 metadata: name of classes and colors of annotations
classes = ["oil_gas"]
colors = [(249, 180, 45)]

# Register dataset and metadata
for d in ["train", "val", "test"]:
    DatasetCatalog.register(d, lambda d=d:
                            get_dataset_dicts(os.path.join("../Satellite", d), annotation_json))
    # Key-value mapping to interpret what’s in the dataset: names of classes, colors of classes
    MetadataCatalog.get(d).thing_classes = classes
    MetadataCatalog.get(d).thing_colors = colors

In [6]:
cfg = get_cfg()
# keep those 2 to avoid errors of MIN_SIZE_TRAIN = 800 and resize not found
#change in config: 
#MIN_SIZE_TRAIN: !!python/tuple
#- 1000
cfg.INPUT.MIN_SIZE_TRAIN = (1000,)
cfg.RESIZE= False
cfg.merge_from_file(conf_path)
cfg.INPUT.MIN_SIZE_TRAIN = (1000,)

cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = filter_empty_annot

print("Configuration setup completed.")

predictor = DefaultPredictor(cfg)
print("Predictor created.")

evaluator = COCOEvaluator(dataset_eval, output_dir=out_dir)
print("Evaluator created.")

val_loader = build_detection_test_loader(cfg, dataset_eval)
print("Validation loader created.")

results = inference_on_dataset(predictor.model, val_loader, evaluator)
print("Inference on dataset completed.")

with open(os.path.join(out_dir, results_file), 'w') as f:
    json.dump(results, f)
print("Results dumped")

Loading config ../Satellite/output/lr001_BS4_oil_gas_test_13/NSOD2cfg_1000_169r_1000pix_noBT_lr001_BS4_oil_gas_test_13.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


Configuration setup completed.
Predictor created.
Evaluator created.
Validation loader created.


/scistor/ivm/fjo101/miniconda3/envs/nso/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/scistor/ivm/fjo101/miniconda3/envs/nso/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.922
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.999
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.852
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.895
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.929
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.875
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.940
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.940
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.867
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.906
 Average Recall     (AR) @[ IoU=0.50:0.

In [7]:
#out_dir = '../NSO/output/lr001_BS4_empty-annot-50/eval'
# Load the JSON pred file into a DataFrame
df_pred = pd.read_json(os.path.join(out_dir, 'coco_instances_results.json'))
df_pred = df_pred[df_pred['score'] > 0.5] 

# Load the JSON ground truth file into a DataFrame

# CHANGE VAL OR TEST
# Extract the "images" key and create a DataFrame from it

# Ground thruth OG
df_truth_og = pd.read_json(os.path.join(os.path.join("../Satellite", dataset_eval), annotation_json))
with open(os.path.join(out_dir, 'test_coco_format.json'), 'r') as j:
     data = json.loads(j.read())
images_data = data.get('images', [])  # Get the list of "images" objects
annotation_data = data.get('annotations', [])  # Get the list of "categories" objects
# Create a DataFrame from the valid "images" objects
df_images = pd.DataFrame(images_data)
df_annotations = pd.DataFrame(annotation_data)
#Drop duplicates: where are they comming from???? =>val_coco is automatically done by D2 check if where nitially in dataset
df_annotations = df_annotations.drop_duplicates(subset=['bbox'])

In [ ]:
df_truth = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how='left')



In [ ]:
# #drop duplicates => why are they there?
#df_truth = df_truth.drop_duplicates(subset=['bbox'])
df_eval = pd.merge(df_truth, df_pred, left_on='id_x', right_on='image_id', how='left')
# df_eval = df_eval.drop_duplicates(subset=['segmentation_x'])
# #draop images with several annotation, just need to keep one
df_eval = df_eval.drop_duplicates(subset=['id_x'])
df_eval['truth'] = None
df_eval['pred'] = None

In [ ]:
df_eval['truth'] = np.where(df_eval.bbox_x.isnull(), 0, 1)
df_eval['pred'] = np.where(df_eval.score.isnull(), 0, 1)

In [ ]:
from sklearn.metrics import confusion_matrix

#ravel to flatten conf matrix to 1D insted of 2D
tn, fp, fn, tp = confusion_matrix(df_eval['truth'], df_eval['pred']).ravel()
precision_score = tp / (tp + fp)
recall_score = tp / (tp + fn)
f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)
print("Precision: ", precision_score)
print("Recall: ", recall_score)
print ("Fscore: ", f1_score)

In [ ]:
# Convert int64 values to native Python int
tn = int(tn)
fp = int(fp)
fn = int(fn)
tp = int(tp)

metrics_dict = {
    'tn': tn,
    'fp': fp,
    'fn': fn,
    'tp': tp,
    'precision_score': precision_score,
    'recall_score': recall_score,
    'f1_score': f1_score
}

# Define the file path
json_metrics = 'metrics_tiles.json'
json_path = os.path.join(out_dir, json_metrics)

# Save the metrics to a JSON file
with open(json_path, 'w') as json_file:
    json.dump(metrics_dict, json_file, indent=4)

print(f'Metrics saved to {json_file}')
print(f1_score)

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(df_eval['truth'], df_eval['pred'])

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix with enhanced visualization
plt.figure(figsize=(10, 8))
sns.set(font_scale=1.4)  # Adjust to increase the size of the labels
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Predicted: Oil & Gas', 'Predicted: Oil & Gas'],
            yticklabels=['Actual: Oil & Gas', 'Actual: Oil & Gas'])
plt.xlabel('Prediction', fontsize=16)
plt.ylabel('Truth', fontsize=16)
plt.title('Confusion Matrix', fontsize=20)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()


In [ ]:
import cv2
import os
import json
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
import random

# Function to load image
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
    return image

# Function to visualize ground truth and predictions
def visualize_predictions(image_path, ground_truth, predictions):
    image = load_image(image_path)
    if image is None:
        return
    
    visualizer = Visualizer(image[:, :, ::-1], MetadataCatalog.get(dataset_eval), scale=1.2)
    
    # Draw ground truth
    if ground_truth is not None:
        for gt in ground_truth:
            bbox = gt['bbox_x']
            # Check if bbox is correctly formatted
            if len(bbox) != 4:
                print(f"Invalid Ground Truth BBox format: {bbox}")
                continue
            bbox_mode = BoxMode.convert(bbox, BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)  # Assuming bbox is in XYWH format
            print(f"Ground Truth BBox: {bbox_mode}")
            visualizer.draw_box(bbox_mode, edge_color=(0, 1, 0, 1))  # RGBA values within 0-1 range
            visualizer.draw_text("GT", bbox_mode[:2], color='g', font_size=10)
    
    # Draw predictions
    if predictions is not None:
        for pred in predictions:
            bbox = pred['bbox_y']
            # Check if bbox is correctly formatted
            if len(bbox) != 4:
                print(f"Invalid Prediction BBox format: {bbox}")
                continue
            bbox_mode = BoxMode.convert(bbox, BoxMode.XYXY_ABS, BoxMode.XYXY_ABS)  # Assuming bbox is already in XYXY format
            score = pred['score']
            print(f"Prediction BBox: {bbox_mode}, Score: {score}")
            visualizer.draw_box(bbox_mode, edge_color=(1, 0, 0, 1))  # RGBA values within 0-1 range
            visualizer.draw_text(f"Pred: {score:.2f}", bbox_mode[:2], color='r', font_size=10)
    
    vis_image = visualizer.get_output().get_image()[:, :, ::-1]
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_image)
    plt.axis('off')
    plt.show()

# Ensure dataset_eval is defined
dataset_eval = "test"  # Change this to the appropriate dataset name if different

# Visualize 10 random examples
image_dir = "../Satellite/test"  # Correct base path for the test images
unique_image_names = df_eval['file_name'].unique()
random_images = random.sample(list(unique_image_names), 10)

for full_image_name in random_images:
    # Get image file path
    image_path = full_image_name
    
    # Print the file path to ensure correctness
    print(f"Image Path: {image_path}")
    
    # Get ground truth and predictions for the specific image using the full path
    gt_info = df_eval[df_eval['file_name'] == full_image_name]
    print(f"Ground truth records found: {len(gt_info)}")
    
    ground_truth = gt_info[~gt_info['bbox_x'].isnull()][['bbox_x', 'category_id_x']].to_dict('records')
    
    # Print ground truth information for debugging
    print("Ground Truth Info:")
    for gt in ground_truth:
        print(gt)
    
    pred_info = df_eval[df_eval['file_name'] == full_image_name]
    print(f"Prediction records found: {len(pred_info)}")
    
    predictions = pred_info[~pred_info['bbox_y'].isnull()][['bbox_y', 'score', 'category_id_y']].to_dict('records')
    
    # Print prediction information for debugging
    print("Prediction Info:")
    for pred in predictions:
        print(pred)
    
    # Visualize predictions on the image
    visualize_predictions(image_path, ground_truth, predictions)


In [ ]:
import cv2
import os
import json
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
import random
import numpy as np
import pycocotools.mask as mask_util

# Function to load image
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
    return image

# Function to convert polygon segmentation to binary mask
def polygon_to_mask(segmentation, height, width):
    mask = np.zeros((height, width), dtype=np.uint8)
    points = np.array(segmentation).reshape(-1, 2)
    cv2.fillPoly(mask, [points], 1)
    return mask

# Function to visualize ground truth and predictions with segmentation masks
def visualize_predictions(image_path, ground_truth, predictions):
    image = load_image(image_path)
    if image is None:
        return
    
    visualizer = Visualizer(image[:, :, ::-1], MetadataCatalog.get(dataset_eval), scale=1.2)
    
    # Draw ground truth masks
    if ground_truth is not None:
        for gt in ground_truth:
            segmentation = gt['segmentation_x']
            print(f"Ground Truth Segmentation: {segmentation}")
            mask = polygon_to_mask(segmentation, image.shape[0], image.shape[1])
            visualizer.draw_binary_mask(mask, color=(0, 1, 0, 0.3), edge_color=(0, 1, 0, 1))
            visualizer.draw_text("GT", (segmentation[0][0], segmentation[0][1]), color='g', font_size=10)
    
    # Draw prediction masks
    if predictions is not None:
        for pred in predictions:
            segmentation = pred['segmentation_y']
            score = pred['score']
            print(f"Prediction Segmentation: {segmentation}, Score: {score}")
            mask = mask_util.decode(segmentation)
            visualizer.draw_binary_mask(mask, color=(1, 0, 0, 0.3), edge_color=(1, 0, 0, 1))
            visualizer.draw_text(f"Pred: {score:.2f}", (mask.shape[1] // 2, mask.shape[0] // 2), color='r', font_size=10)
    
    vis_image = visualizer.get_output().get_image()[:, :, ::-1]
    plt.figure(figsize=(10, 10))
    plt.imshow(vis_image)
    plt.axis('off')
    plt.show()

# Ensure dataset_eval is defined
dataset_eval = "test"  # Change this to the appropriate dataset name if different

# Visualize 10 random examples
image_dir = "../Satellite/test"  # Correct base path for the test images
unique_image_names = df_eval['file_name'].unique()
random_images = random.sample(list(unique_image_names), 10)

for full_image_name in random_images:
    # Get image file path
    image_path = full_image_name
    
    # Print the file path to ensure correctness
    print(f"Image Path: {image_path}")
    
    # Get ground truth and predictions for the specific image using the full path
    gt_info = df_eval[df_eval['file_name'] == full_image_name]
    print(f"Ground truth records found: {len(gt_info)}")
    
    ground_truth = gt_info[~gt_info['segmentation_x'].isnull()][['segmentation_x', 'category_id_x']].to_dict('records')
    
    # Print ground truth information for debugging
    print("Ground Truth Info:")
    for gt in ground_truth:
        print(gt)
    
    pred_info = df_eval[df_eval['file_name'] == full_image_name]
    print(f"Prediction records found: {len(pred_info)}")
    
    predictions = pred_info[~pred_info['segmentation_y'].isnull()][['segmentation_y', 'score', 'category_id_y']].to_dict('records')
    
    # Print prediction information for debugging
    print("Prediction Info:")
    for pred in predictions:
        print(pred)
    
    # Visualize predictions on the image
    visualize_predictions(image_path, ground_truth, predictions)



In [ ]:
# Filter for overlaps where truth and pred are both 1
overlaps_truth_pred_1 = df_eval[(df_eval['truth'] == 1) & (df_eval['pred'] == 1)]

# Filter for non-overlaps where truth is 1 and pred is 0
non_overlaps_truth_1_pred_0 = df_eval[(df_eval['truth'] == 1) & (df_eval['pred'] == 0)]

# Filter for truth == 0 and pred == 1
truth_0_pred_1 = df_eval[(df_eval['truth'] == 0) & (df_eval['pred'] == 1)]

# Filter for truth == 0 and pred == 0
truth_0_pred_0 = df_eval[(df_eval['truth'] == 0) & (df_eval['pred'] == 0)]

# Extract unique image IDs for overlaps (truth == 1 and pred == 1)
overlap_image_ids = overlaps_truth_pred_1['file_name'].unique()

# Extract unique image IDs for non-overlaps (truth == 1 and pred == 0)
non_overlap_image_ids = non_overlaps_truth_1_pred_0['file_name'].unique()

# Extract unique image IDs for truth == 0 and pred == 1
truth_0_pred_1_image_ids = truth_0_pred_1['file_name'].unique()

# Extract unique image IDs for truth == 0 and pred == 0
truth_0_pred_0_image_ids = truth_0_pred_0['file_name'].unique()

# Print the results with counts
print("Image IDs where truth and prediction overlap (truth == 1 and pred == 1):")
print(overlap_image_ids)
print(f"Count: {len(overlap_image_ids)}\n")

print("Image IDs where truth is 1 and prediction is 0:")
print(non_overlap_image_ids)
print(f"Count: {len(non_overlap_image_ids)}\n")

print("Image IDs where truth is 0 and prediction is 1:")
print(truth_0_pred_1_image_ids)
print(f"Count: {len(truth_0_pred_1_image_ids)}\n")

print("Image IDs where truth is 0 and prediction is 0:")
print(truth_0_pred_0_image_ids)
print(f"Count: {len(truth_0_pred_0_image_ids)}\n")


In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import pycocotools.mask as mask_util
import cv2

# Function to convert polygon segmentation to binary mask
def polygon_to_mask(segmentation, height, width):
    mask = np.zeros((height, width), dtype=np.uint8)
    points = np.array(segmentation).reshape(-1, 2)
    cv2.fillPoly(mask, [points], 1)
    return mask

# Function to plot a single image with annotations
def plot_single_image(tile_path, annotation_path, df_eval, ax, title, is_prediction=False):
    # Load the image
    image = Image.open(tile_path)
    
    # Load the annotation JSON file
    with open(annotation_path, 'r') as f:
        annotations = json.load(f)
    
    # Extract the filename from the tile path
    filename = tile_path.split('/')[-1]
    
    # Get the annotations for the specified tile
    tile_annotations = annotations.get(filename, {}).get('regions', {})
    
    # Get predictions from df_eval
    predictions = df_eval[df_eval['file_name'] == tile_path].to_dict('records')
    
    ax.imshow(image)
    ax.set_title(title)
    
    # Function to plot regions
    def plot_regions(ax, regions, edgecolor, facecolor, label_suffix):
        for region in regions.values():
            shape_attributes = region.get('shape_attributes', {})
            region_attributes = region.get('region_attributes', {})
            
            if shape_attributes.get('name') == 'polygon':
                all_points_x = shape_attributes.get('all_points_x', [])
                all_points_y = shape_attributes.get('all_points_y', [])
                polygon = list(zip(all_points_x, all_points_y))
                
                # Determine color based on type
                annotation_type = region_attributes.get('type', 'unknown')
                if annotation_type == 'oil/gas':
                    edgecolor = 'g'
                    facecolor = 'green'
                else:
                    edgecolor = 'r'
                    facecolor = 'red'
                
                # Create a polygon patch with a semi-transparent fill color
                poly_patch = patches.Polygon(polygon, closed=True, edgecolor=edgecolor, facecolor=facecolor, alpha=0.5)
                ax.add_patch(poly_patch)
                
                # Get the centroid of the polygon to place the label
                centroid_x = sum(all_points_x) / len(all_points_x)
                centroid_y = sum(all_points_y) / len(all_points_y)
                
                # Get the type/category for the annotation
                annotation_type = region_attributes.get('type', 'unknown')
                
                # Add text annotation to the plot
                ax.text(centroid_x, centroid_y, f"{annotation_type} {label_suffix}", color='white', fontsize=12, ha='center', va='center',
                        bbox=dict(facecolor=edgecolor, alpha=0.5, edgecolor='none', pad=1))
    
    # Plot ground truth annotations
    if not is_prediction:
        plot_regions(ax, tile_annotations, edgecolor='r', facecolor='red', label_suffix='(GT)')
    
    # Plot predictions from df_eval
    if is_prediction:
        for pred in predictions:
            if 'segmentation_y' in pred and pred['segmentation_y']:
                segmentation = pred['segmentation_y']
                score = pred['score']
                if isinstance(segmentation, dict) and 'size' in segmentation and 'counts' in segmentation:  # Check if it's a RLE
                    mask = mask_util.decode(segmentation)
                    poly_patch = patches.Polygon(np.argwhere(mask).reshape(-1, 2), closed=True, edgecolor='b', facecolor='blue', alpha=0.5)
                    ax.add_patch(poly_patch)
                    centroid_x, centroid_y = np.mean(np.argwhere(mask), axis=0)
                else:
                    # Handle the polygon segmentation case
                    if isinstance(segmentation, list):
                        segmentation = np.array(segmentation).reshape(-1, 2)
                        poly_patch = patches.Polygon(segmentation, closed=True, edgecolor='b', facecolor='blue', alpha=0.5)
                        ax.add_patch(poly_patch)
                        centroid_x = np.mean(segmentation[:, 0])
                        centroid_y = np.mean(segmentation[:, 1])
                    else:
                        continue
                # Add text annotation to the plot
                ax.text(centroid_x, centroid_y, f"Pred: {score:.2f}", color='white', fontsize=12, ha='center', va='center',
                        bbox=dict(facecolor='blue', alpha=0.5, edgecolor='none', pad=1))

# Example data
image_paths_overlap = [
    '../Satellite/test/191_Alborg_2000_4000.png',
 '../Satellite/test/248_Split_2000_2000.png',
 '../Satellite/test/28_Reka Luga_19000_9000.png',
 '../Satellite/test/260_Cardiff_2000_2000.png',
 '../Satellite/test/102_Odesa_4000_2000.png',
 '../Satellite/test/320_Port Jerome_3000_9000.png',
 '../Satellite/test/262_Sundsvall_10000_1000.png',
 '../Satellite/test/12_Bukhta Vanino_2000_3000.png',
 '../Satellite/test/349_Vlores_18000_6000.png',
 '../Satellite/test/73_La Pallice_0_3000.png',
 '../Satellite/test/361_Teesport_5000_2000.png',
 '../Satellite/test/183_Antwerp_17000_5000.png',
 '../Satellite/test/293_Port of Le Havre_2000_13000.png',
 '../Satellite/test/106_Hamina_7000_1000.png',
 '../Satellite/test/107_Batumi_0_3000.png',
 '../Satellite/test/349_Vlores_12000_15000.png',
 '../Satellite/test/281_Soudha_0_0.png',
 '../Satellite/test/349_Vlores_15000_17000.png',
 '../Satellite/test/105_Port-De-Bouc_2000_0.png',
 '../Satellite/test/151_Vlissingen_2000_12000.png',
 '../Satellite/test/186_Ghent_22000_7000.png',
 '../Satellite/test/303_Karsto_0_0.png',
 '../Satellite/test/201_Port Taman_3000_0.png',
 '../Satellite/test/308_Dublin_2000_2000.png',
 '../Satellite/test/191_Alborg_1000_5000.png',
 '../Satellite/test/374_Mersin_2000_1000.png',
 '../Satellite/test/316_Durres_0_1000.png',
 '../Satellite/test/301_Rotterdam_18000_43000.png',
 '../Satellite/test/201_Port Taman_1000_1000.png',
 '../Satellite/test/88_Stockholm_3000_2000.png',
 '../Satellite/test/86_Terneuzen_1000_0.png',
 '../Satellite/test/260_Cardiff_3000_0.png',
 '../Satellite/test/301_Rotterdam_9000_21000.png',
 '../Satellite/test/81_Porto Di Corsini_9000_1000.png',
 '../Satellite/test/90_Swinoujscie_4000_0.png',
 '../Satellite/test/183_Antwerp_16000_17000.png',
 '../Satellite/test/123_Iskenderun_0_1000.png',
 '../Satellite/test/183_Antwerp_19000_17000.png',
 '../Satellite/test/229_Hamburg_5000_8000.png',
 '../Satellite/test/313_Koper_1000_0.png',
 '../Satellite/test/308_Dublin_1000_0.png',
 '../Satellite/test/64_Bakar_2000_2000.png',
 '../Satellite/test/308_Dublin_0_2000.png',
 '../Satellite/test/229_Hamburg_7000_7000.png',
 '../Satellite/test/222_Izhevskoye_1000_9000.png',
 '../Satellite/test/369_Setubal_7000_15000.png',
 '../Satellite/test/293_Port of Le Havre_2000_9000.png',
 '../Satellite/test/281_Soudha_1000_0.png',
 '../Satellite/test/309_Muuga-Port of Talinn_2000_0.png',
 '../Satellite/test/356_Rostock_5000_9000.png',
 '../Satellite/test/376_Brindisi_0_3000.png',
 '../Satellite/test/188_Helsingborg_3000_0.png',
 '../Satellite/test/301_Rotterdam_14000_26000.png',
 '../Satellite/test/229_Hamburg_8000_11000.png',
 '../Satellite/test/60_Port Saint Louis Du Rhone_2000_3000.png',
 '../Satellite/test/350_Kingston Upon Hull_1000_5000.png',
 '../Satellite/test/293_Port of Le Havre_4000_0.png',
 '../Satellite/test/210_Gdynia_3000_8000.png',
 '../Satellite/test/161_Fawley Marine Terminal_0_7000.png',
 '../Satellite/test/259_Drammen_3000_0.png',
 '../Satellite/test/209_Porto Di Lido-Venezia_7000_3000.png',
]

image_paths_truth_1_pred_0 = [
    '../Satellite/test/349_Vlores_20000_12000.png',
     '../Satellite/test/258_Lulea_5000_5000.png',
     '../Satellite/test/349_Vlores_16000_5000.png',
     '../Satellite/test/28_Reka Luga_13000_15000.png',
     '../Satellite/test/374_Mersin_3000_2000.png',
     '../Satellite/test/206_Umea Hamn_3000_2000.png',
     '../Satellite/test/82_Dunkerque Port Ouest_6000_1000.png',
     '../Satellite/test/117_Barry_1000_1000.png',
     '../Satellite/test/183_Antwerp_12000_2000.png',
     '../Satellite/test/247_Murmansk_36000_3000.png',
     '../Satellite/test/304_Vostochnyy_6000_0.png',
     '../Satellite/test/140_Primorsk_4000_3000.png',
     '../Satellite/test/307_Aliaga_6000_3000.png',
     '../Satellite/test/75_Sines_5000_10000.png',
     '../Satellite/test/229_Hamburg_7000_8000.png',
     '../Satellite/test/234_Shoreham Harbour_0_2000.png',
     '../Satellite/test/28_Reka Luga_20000_9000.png',
     '../Satellite/test/175_Dudinka_1000_1000.png',
]

image_paths_truth_0_pred_1 = [
    '../Satellite/test/106_Hamina_6000_2000.png',
]

annotation_path = '../Satellite/test/via_region_data_with_empty_annotations.json'

# Plot images
def plot_images_by_category(image_paths, annotation_path, df_eval, category_title):
    n_images = len(image_paths)
    fig, axs = plt.subplots(n_images, 2, figsize=(20, 10 * n_images))
    if n_images == 1:
        axs = [axs]  # Ensure axs is iterable if there's only one image

    for i, image_path in enumerate(image_paths):
        plot_single_image(image_path, annotation_path, df_eval, axs[i][0], f"{category_title} - Ground Truth", is_prediction=False)
        plot_single_image(image_path, annotation_path, df_eval, axs[i][1], f"{category_title} - Predictions", is_prediction=True)
    
    plt.show()

# Plot each category
plot_images_by_category(image_paths_overlap, annotation_path, df_eval, 'True Positive')
plot_images_by_category(image_paths_truth_1_pred_0, annotation_path, df_eval, 'False Negative')
plot_images_by_category(image_paths_truth_0_pred_1, annotation_path, df_eval, 'False Positive')